# Primeiros Passos na Segmentação de Imagens Ortomosaicas

## 1. Segmentação das Imagens
O primeiro passo consiste em dividir as imagens ortomosaicas em várias imagens menores de 600x600 pixels, separando todas as camadas (R, G, B) individualmente. Essa etapa é essencial para alimentar o modelo corretamente e otimizar o processamento das imagens.

## 2. Importação de Modelos Pré-treinados e Dataset
Após a segmentação, é necessário importar os modelos pré-treinados e o dataset para realizar inferências (predições). Utilizaremos o modelo **DeepLabV3Plus** com **ResNet-50** treinado para classificação de três classes: **fundo, milho e daninha**.

Repositório do modelo: [DeepLabV3Plus-PyTorch](https://github.com/VainF/DeepLabV3Plus-Pytorch)

### 2.1 Análise e Recorte do Dataset
Aqui entra uma parte mais complexa: precisamos analisar o dataset segmentado anteriormente e garantir que nossas imagens estejam devidamente rotuladas. Ferramentas úteis para essa etapa incluem:
- **LabelMe**
- **CUAT**
- **VGG Image Annotator (VIA)**

Essas ferramentas ajudam a criar e validar anotações para treinamento do modelo.

## 3. Fine-tuning ou Transfer Learning
Com as imagens devidamente segmentadas e anotadas, podemos prosseguir para a etapa de **fine-tuning** ou **transfer learning**. Esse processo ajusta o modelo pré-treinado para obter melhor desempenho na segmentação de milho e plantas daninhas, refinando os pesos da rede neural com nosso próprio dataset.

Essa abordagem permite que o modelo se adapte às características específicas das nossas imagens, melhorando sua precisão na segmentação.



### Para atingir esse objetivo, usei Python juntamente com a biblioteca Pillow, que captura uma a uma das imagens .tif e as recorta em blocos de 600x600. Em seguida, ignora as imagens com mais de 10% de alpha e as separa nas camadas RGB, gerando os arquivos nome_imagemR.png, nome_imagemG.png, nome_imagemB.png. As imagens .tif estão localizadas no diretório /imgs/tifs, e as imagens .png são salvas em /imgs/pngs.

In [1]:
#Atualizar dependencias
!pip freeze > requirements.txt

In [ ]:
#Instalar dependencias 
!pip install -r requirements.txt

In [ ]:

import os
from PIL import Image
import numpy as np

# Diretórios
input_dir = '/imgs/tifs'  # Diretório com as imagens .tif
output_dir = '/imgs/pngs'  # Diretório para salvar as imagens .png

# Função para verificar o alpha da imagem
def has_high_alpha(img, threshold=0.1):
    # Obtém o canal alpha, se existir
    alpha = np.array(img.split()[3]) if img.mode == 'RGBA' else None
    if alpha is not None:
        # Calcula a porcentagem de pixels com alpha > 10%
        alpha_percentage = np.sum(alpha > 25) / alpha.size  # Ajuste o valor 25 conforme necessário
        return alpha_percentage > threshold
    return False

# Função para salvar as imagens RGB separadas
def save_rgb_layers(img, base_name):
    # Divide a imagem em suas camadas RGB
    r, g, b = img.split()
    
    # Salva cada camada como uma imagem separada
    r.save(os.path.join(output_dir, f"{base_name}R.png"))
    g.save(os.path.join(output_dir, f"{base_name}G.png"))
    b.save(os.path.join(output_dir, f"{base_name}B.png"))

# Função para processar as imagens
def process_images():
    # Cria o diretório de saída se não existir
    os.makedirs(output_dir, exist_ok=True)

    # Itera sobre todas as imagens no diretório de entrada
    for filename in os.listdir(input_dir):
        if filename.endswith('.tif'):
            file_path = os.path.join(input_dir, filename)
            img = Image.open(file_path)
            
            # Ignora imagens com mais de 10% de alpha
            if has_high_alpha(img):
                print(f"Ignorando {filename} devido ao alto alpha.")
                continue

            # Processa a imagem em blocos de 600x600
            img_width, img_height = img.size
            for i in range(0, img_width, 600):
                for j in range(0, img_height, 600):
                    # Define as coordenadas do bloco
                    box = (i, j, min(i + 600, img_width), min(j + 600, img_height))
                    cropped_img = img.crop(box)
                    
                    # Cria um nome base para o arquivo
                    base_name = f"{filename.split('.')[0]}_{i}_{j}_"
                    
                    # Salva as camadas RGB separadas
                    save_rgb_layers(cropped_img, base_name)

# Executa o processamento
if __name__ == "__main__":
    process_images()
